In [1]:
from torch_geometric import nn
from torch_geometric import utils
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
import torch.nn
import torch.optim as optim
import numpy as np
import os
import time
from tqdm import tqdm
from scipy.sparse import coo_matrix
# from tqdm.notebook import tqdm

# from models.encoder import Encoder, EncoderLayer, ConvLayer, EncoderStack
# from models.attn import FullAttention, ProbAttention, AttentionLayer

seed = 123456789
np.random.seed(seed)

torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [9]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
output = F.cosine_similarity(input1, input2, dim=1)
print(output.shape)

torch.Size([100])


In [13]:
device = 'cuda'
market_name = 'NASDAQ'
# data_path = 'D:/data/qt/data/2013-01-01'
data_path = 'D:/data/qt/data/rr8_volumn'
col_train = ['rr', 'rr5', 'rr10', 'rr20', 'rr30', 'volumn']
col_label = ['rr-1', 'rr-5', 'rr-30']
ecod_in = len(col_train)
tickers = market_name + '_tickers_qualify_dr-0.98_min-5_smooth.csv'
tickers = np.genfromtxt(os.path.join(data_path, '..', tickers),
                                     dtype=str, delimiter='\t', skip_header=False)
inci_mat = np.load('hg_test.npy')
hyp_input = torch.from_numpy(inci_mat).to(device)

FileNotFoundError: [Errno 2] No such file or directory: 'hg_test.npy'

In [11]:
# generate random array and tensor equivalent
x = torch.from_numpy(np.random.rand(1025, 3))
y = torch.from_numpy(np.random.rand(1025, 3))

# compute corrcoef using those functions
z = torch.corrcoef(torch.cat([x[:, 1].unsqueeze(0), y[:, 1].unsqueeze(0)]))

z

tensor([[ 1.0000, -0.0212],
        [-0.0212,  1.0000]], dtype=torch.float64)

In [8]:
torch.cat([x.unsqueeze(0), y.unsqueeze(0)]).shape

torch.Size([2, 1025, 3])

In [9]:
np_test = np.load('./training/hg_test.npy')

In [11]:
np_test[1].max().item() + 1

2518

In [9]:
industry = r"C:\Users\kikyo\code\qt\sthan-sr-aaai-main\training\NASDAQ_industry_relation.npy"
wiki = '/home/kikyo/data/qt/relation/wikidata/NASDAQ_wiki_relation_fill.npy'
npy = np.load(wiki)

In [10]:
npy.shape

(1026, 1026, 43)

In [11]:
row = []
col = []
edge = -1
for i in tqdm(range(43)):         
    local = utils.from_scipy_sparse_matrix(coo_matrix(npy[:, :, i]))
    node0 = 0
    for j in range(len(local[0][0])):
        if local[0][0][j] != node0:
            node0 = local[0][0][j]
            row.append(node0)
            edge += 1
        else:
            row.append(local[0][1][j])
        col.append(edge)
hg = np.array([np.array(row), col])
np.save('/home/kikyo/code/hidden-conc-hypg/hg_test.npy', hg)

100%|██████████| 43/43 [00:01<00:00, 25.84it/s]


In [8]:
hg[1].shape

(401304,)

In [74]:
np.array(row).shape

(159188,)

In [3]:
edge_index = torch.tensor([
    [0, 1, 1, 2, 2, 3],
    [1, 0, 2, 1, 3, 2],
])
# print(utils.to_scipy_sparse_matrix(edge_index))
print(coo_matrix(edge_index))

  (0, 1)	1
  (0, 2)	1
  (0, 3)	2
  (0, 4)	2
  (0, 5)	3
  (1, 0)	1
  (1, 2)	2
  (1, 3)	1
  (1, 4)	3
  (1, 5)	2


In [37]:
test = np.array([[0, 1, 1],[0, 0, 1], [1, 1 ,0]])
utils.from_scipy_sparse_matrix(coo_matrix(test))

(tensor([[0, 0, 1, 2, 2],
         [1, 2, 2, 0, 1]]),
 tensor([1, 1, 1, 1, 1], dtype=torch.int32))